# RESDecryptor

This notebook helps in discovering passwords and encryption mechanisms in RESAM. The RESDectyptor module contains some functions that do this but for developing / playing around I find this easier than a (jupyter) console.

The simplest encryption mechanism in RESAM is a substituion cypher (Vigenere) which is used at the task level. Some tasks allow you to set a security context and the UI offers to store this for future use. Different keys are used for different task types but consistency is, as usual, not to be found.

The building block xml seems to have evolved over time, with many little (annoying) differences. Most of teh time the password for the security context is stored in `//tasks/task/settings/password` but this can also be something like `//tasks/task/settings/dbpassword` or `//tasks/task/settings/securitypassword`.

The xpath expression I use for now is  `//*[contains(local-name(), 'password')]` which seems to catch them all (for now).

In [1]:
from resdecryptor import *

somepw = 'abcdefghABCDEFGH1234!@#$'

cyphertexts = [
    '00D000CF00D000C500D300CA00AA00D700AE00AF00A400B200A9008900B600B5009E009300A10098006400AF00900091',
    '00B200AE00B600C700D700CF00D700DC00940093008F009700A800B800B000B800A5008500840080007400A30095008D',
    '00C600D000C700A400B200C700D000D4009400A700B100A80085009300A800B1009D0085009800A20085008000700085',
    '00B400AA00A600D300D200D300C800D600A500B500960097008D008900B600B5009E009300A10098009400930076006C',
    '00C600C400B600C900D700DC00D000CB00A6009900A800A6009800AB00B900BE009A00950098008B008600A200760089'
    ]

In [2]:
for t in cyphertexts:
    print(derive_key(somepw, t))

ommandCommandCommandComm
QLScriptSQLScriptSQLScri
end@MailSend@MailSend@Ma
SHCommandsSSHCommandsSSH
ebServiceWebServiceWebSe
